# Running commands on multiple samples

Now, fair warning, you're going to wish we'd told you this earlier on. However, then you wouldn't have had the fun of running and updating each of the previous commands, growling at typos and generally wishing that you'd gone for that cup of coffee before starting this tutorial.

Here we go....we can use a **loop** to run the same commands for multiple samples.

There's a great introduction to bash scripting and loops as part of our **Unix module**. But let's take a look at how we could have generated genome alignments for all of our samples using a single loop.

Whenever you write a loop, it's always a good idea to build it up slowly to check that it's doing what you think.

In [ ]:
for r in data/*.fastq.gz
do
  echo $r
done  

This loop looks for all (*) files which end with ".fastq.gz". The for loop then executes a sequence of commands for each file name that it finds. In the first iteration its "data/MT1_1.fastq.gz", then "data/MT1_2.fastq.gz" and so on... In each iteration, we assigned each filename that it found to a variable called "r". 

`for r in *.fastq.gz`

Then, to check we got what we expected, we printed what the variable "r" represented back to the terminal. Because we want to use the variable ("r") we created we need to use dollar ($) symbol.

`echo $r`

Now, if we left things as they are, we would be running the commands twice for each sample. This is because we have two FASTQ files for each sample i.e. "_1.fastq.gz" and "_2.fastq.gz".  Let's change our loop so that we only get the "_1.fastq.gz" files.

In [ ]:
for r1 in data/*_1.fastq.gz
do
  echo $r1
done  

Great! Now, the only problem here is that we're going to want to use both the "_1.fastq.gz" and the "_2.fastq.gz" files in our mapping. We can get around this by removing the "data/" directory and "_1.fastq.gz" suffix from the filename to give us our sample name.

`sample=$(basename $r1)`
`sample=${sample/_1.fastq.gz/}`

This will get the base filename (e.g. "MT1_1.fastq.gz") and replace the "_1.fastq.gz" at the end of the filename we stored as "r1" with nothing. 

We've added a little descriptive message so that when we run our loop we know which iteration it's on and what it's doing. Let's try adding our HISAT2 mapping command.

_Note: we assume that the HISAT2 index has already been generated as that's a command you'll only need to run once._

In [ ]:
for r1 in data/*_1.fastq.gz
do
  sample=$(basename $r1)
  sample=${sample/_1.fastq.gz/}
  echo "Processing sample: "$sample
  
  echo "Mapping sample: "$sample
  hisat2 --max-intronlen 10000 -x data/PccAS_v3_hisat2.idx \
  -1 "data/${sample}_1.fastq.gz" -2 "data/${sample}_2.fastq.gz" -S "data/${sample}.sam"
done

Notice that because we're using a variable as part of the filename, we need to write the filename in double quotes.

`data/${sample}_1.fastq.gz`

Now let's add in our `samtools` commands.

In [ ]:
for r1 in data/*_1.fastq.gz
do
  sample=$(basename $r1)
  sample=${sample/_1.fastq.gz/}
  echo "Processing sample: "$sample
  
  echo "Mapping sample: "$sample
  hisat2 --max-intronlen 10000 -x data/PccAS_v3_hisat2.idx \
  -1 "data/${sample}_1.fastq.gz" -2 "data/${sample}_2.fastq.gz" -S "data/${sample}.sam"
  
  echo "Converting SAM to BAM: "$sample
  samtools view -b -o "data/${sample}.bam" "data/${sample}.sam"
  
  echo "Sorting BAM: "$sample
  samtools sort -o "data/${sample}_sorted.bam" "data/${sample}.bam" 
  
  echo "Indexing BAM: "$sample
  samtools index "data/${sample}_sorted.bam"
done

Finally, we don't really want to keep intermediate SAM and unsorted BAM files if we don't have to. They just take up precious space. So, let's make our samtools command a one-liner, passing the stdout from one command to another.

In [ ]:
for r1 in data/*_1.fastq.gz
do
  sample=$(basename $r1)
  sample=${sample/_1.fastq.gz/}
  echo "Processing sample: "$sample
  hisat2 --max-intronlen 10000 -x data/PccAS_v3_hisat2.idx \
  -1 "data/${sample}_1.fastq.gz" -2 "data/${sample}_2.fastq.gz" \
  | samtools view -b - \
  | samtools sort -o "data/${sample}_sorted.bam" - \
  && samtools index "data/${sample}_sorted.bam" 
done

You could also have used this approach for transcript quantification with Kallisto, assuming you had already generated the Kallisto index.

In [ ]:
for r1 in data/*_1.fastq.gz
do
  sample=$(basename $r1)
  sample=${sample/_1.fastq.gz/}
  echo "Quantifying transcripts for sample: "$sample
  kallisto quant -i data/PccAS_v3_kallisto -o "data/${sample}" -b 100 \
  "data/${sample}_1.fastq.gz" "data/${sample}_2.fastq.gz"
done

## Taking a closer look at the SBP genome mapping bash script

In the genome mapping section of this tutorial, we mentioned that the sorted genome alignments had been provided for the three SBP samples and that to generate them, we had run a bash script.

To take a look at the script you can run:

```
less data/map_SBP_samples.sh
```

The script contains commands to run the mapping, converting, sorting and indexing for all of the SBP samples. There's a great introduction to bash scripting and loops in your **Unix module**.

First, the bash script looks for all files in the data directory which start with "SBP" and end with "_1.fastq.gz". This is so that we get one filename per sample.

```
data/SBP*_1.fastq.gz
```

To run the commands for each of our SBP samples: SBP1, SBP2 and SBP3, the script uses a **for loop**. Often, scripts like these can take a while to run and it can be difficult to track what's going on if there is limited or indistinguisable output. Here, we are printing the file path that gets returned by our search.

```
for r1 in data/SBP*_1.fastq.gz
do
  echo $r1
done
```

This will print out:

```
SBP1_1.fastq.gz
SBP2_1.fastq.gz
SBP3_1.fastq.gz
```

Next, the script removes parts of the filename to get the name of the sample it belongs to. It does this becuase both FASTQ files (r1 and r2) are required to align each sample. There are many different ways to do this. This is one example:

```
for r1 in data/SBP*_1.fastq.gz
do
  echo $r1
  sample=$(basename $r1)
  sample=${sample/_1.fastq.gz/}
  echo "Processing sample: "$sample
done
```

Which will print out:

```
Processing sample: SBP1
Processing sample: SBP2
Processing sample: SBP3
```

Finally, the script runs the single command we were using above for the sample:

```
hisat2 --max-intronlen 10000 -x data/PccAS_v3_hisat2.idx \
  -1 "data/${sample}_1.fastq.gz" -2 "data/${sample}_2.fastq.gz" \
  | samtools view -b - \
  | samtools sort -o "data/${sample}_sorted.bam" - \
  && samtools index "data/${sample}_sorted.bam" 
```

Note, when it extracted the sample name in the commands above, it stored it as a variable `$sample`. It can then use the `$sample` variable to create a dynamic command which will run for any of the samples.